# Rubeus Hack Tool
Detects command line parameters used by Rubeus hack tool

## Rule Content
```
- title: Rubeus Hack Tool
  id: 7ec2c172-dceb-4c10-92c9-87c1881b7e18
  description: Detects command line parameters used by Rubeus hack tool
  author: Florian Roth
  references:
  - https://www.harmj0y.net/blog/redteaming/from-kekeo-to-rubeus/
  date: 2018/12/19
  tags:
  - attack.credential_access
  - attack.t1003
  - attack.s0005
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '* asreproast *'
      - '* dump /service:krbtgt *'
      - '* kerberoast *'
      - '* createnetonly /program:*'
      - '* ptt /ticket:*'
      - '* /impersonateuser:*'
      - '* renew /ticket:*'
      - '* asktgt /user:*'
      - '* harvest /interval:*'
    condition: selection
  falsepositives:
  - unlikely
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\ asreproast\ * OR *\ dump\ \/service\:krbtgt\ * OR *\ kerberoast\ * OR *\ createnetonly\ \/program\:* OR *\ ptt\ \/ticket\:* OR *\ \/impersonateuser\:* OR *\ renew\ \/ticket\:* OR *\ asktgt\ \/user\:* OR *\ harvest\ \/interval\:*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()